In [2]:
import pandas as pd
import numpy as np

# Replace 'metrics_results.csv' with the path to your CSV file
file_path = 'metrics_results_maxIters_1000.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify
df.head()


,Run,Best Value,Best Iteration,Total Accepted Moves,Total Rejected Moves,Total Distance Traveled,Total Deviation from Optimum,Distinct Positions Visited,End-to-End Distance,Average Move Distance,Standard Deviation Objective,Acceptance Rate,Best Solution Improvements,Total Function Value Change,Exploration-Exploitation Ratio
0,1,0.0,550,947,54,1003.999133,222277.0,353,50.447993,1.060189,445.536971,0.946054,69,12449.0,1.120643
1,2,0.0,382,911,90,952.330086,292544.0,370,47.423623,1.045368,450.013218,0.910090,68,14616.0,1.148642
2,3,0.0,662,905,96,946.815367,323843.0,366,47.539457,1.046205,480.104020,0.904096,75,15872.0,1.157183
3,4,0.0,546,899,102,971.099639,469106.0,342,56.639209,1.080200,671.571845,0.898102,76,18722.0,1.202759
4,5,17.0,407,919,82,969.928066,344403.0,418,60.108236,1.055417,619.649409,0.918082,73,16682.0,1.149589
